In [194]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [195]:
with open("articles.txt", "r") as file:
  articles = file.read()

tokens = nltk.tokenize.word_tokenize(articles)


In [196]:
def createDBTester(archive):
    testSituations = []
    file = open(archive, "r")
    for row in file:
      right, wrong = row.split()
      testSituations.append((right, wrong))
    file.close()
    return testSituations

In [197]:
def extractOnlyWords(tokens):
  words = []
  for token in tokens:
      if token.isalpha():
        words.append(token.lower())
  return words

words = extractOnlyWords(tokens)
print("Words amount: ", len(words))

Words amount:  393914


In [198]:
letters = "abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç"

def addLetter(wrongWord):
  newWord = []
  for left, right in wrongWord:
    for letter in letters:
      newWord.append(left + letter + right)
  return newWord

def deleteCaracter(wrongWord):
  newWord = []
  for left, right in wrongWord:
    newWord.append(left + right[1:])
  return newWord

def invertLetter(wrongWord):
  newWord = []
  for left, right in wrongWord:
    if len(right) > 1:
      newWord.append(left + right[1] + right[0] + right[2:])
  return newWord

def tradeLetter(wrongWord):
  newWord = []
  for left, right in wrongWord:
    for letter in letters:
      newWord.append(left + letter + right[1:])
  return newWord

def possibilitiesFixWrongWord(wrongWord): 
  fragmentsWord = []
  for i in range(len(wrongWord)+1):
    fragmentsWord.append((wrongWord[:i],wrongWord[i:]))
  possibilitiesWords = addLetter(fragmentsWord)
  possibilitiesWords += deleteCaracter(fragmentsWord)
  possibilitiesWords += tradeLetter(fragmentsWord)
  possibilitiesWords += invertLetter(fragmentsWord)
  return possibilitiesWords



In [199]:
frequency = nltk.FreqDist(words)
totalWords = len(set(words))
wellKnownWords = set(words)

def probability(possibleWord):
  return frequency[possibleWord]/totalWords

def segundBrokerStep(possibilitiesWords):
  newWords = []
  for word in possibilitiesWords:
    newWords += possibilitiesFixWrongWord(word)
  return newWords


In [200]:

def accuracy(broker, testSituations, vocabulary):
  hit = 0
  unknownWords = 0
  amountTry = len(testSituations)
  transformPorcentage = 100
  for rightWord, wrongWord in testSituations:
    fixWord = broker(wrongWord)
    unknownWords += (rightWord not in vocabulary)
    if fixWord == rightWord:
      hit += 1
  accuracy = round(hit*transformPorcentage/amountTry, 2)
  taxUnknownWords = round(unknownWords*transformPorcentage/amountTry, 2)
  return f"Acurrecy {accuracy}% of {amountTry} words. Unknown words: {taxUnknownWords}%"



In [201]:
def brokerV1(word):
  return max(possibilitiesFixWrongWord(word), key=probability)

def brokerV2(writeWord):
  allWords = set(segundBrokerStep(possibilitiesFixWrongWord(writeWord)) + possibilitiesFixWrongWord(writeWord))
  candidates = [writeWord]
  for word in allWords:
    if word in wellKnownWords:
      candidates.append(word)
  return max(candidates, key=probability)



## Change the name of the variable below to the word you would like to correct:


In [202]:
writeWord = "lgiica"

In [203]:
print("brokerV1 result:", brokerV1(writeWord))
accuracy(brokerV1, createDBTester("words.txt"), wellKnownWords)


brokerV1 result: algiica


'Acurrecy 76.34% of 186 words. Unknown words: 6.99%'

In [204]:
print("brokerV2 result: ", brokerV2(writeWord))
accuracy(brokerV2, createDBTester("words.txt"), wellKnownWords)


brokerV2 result:  lógica


'Acurrecy 55.91% of 186 words. Unknown words: 6.99%'